In [ ]:
import sys
sys.path.append("../")

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import os

In [ ]:
import Misc.env_vars as ev
from Logic.base_queries import run_stmt
import Misc.env_vars as ev
from Logic import base_queries as bq

In [ ]:
from Models.models import (od_dq_base, dq_agg_view, dq_agg_sum, dq_col_sum, dq_dim_order_status, dq_dim_sellers, ds_data_sanity, dq_agg_order_stats,
ds_last_run_date)

In [ ]:
from models import engine

In [ ]:
from sqlalchemy import Select, func, or_, desc, and_, asc

In [ ]:
cols_dict = {
    "curr_date": "Current Date",
    "ord_date": "Order Date",
    "seller_np": "Seller NP",
    "null_fulfilment_id": "Fulfillment ID",
    "null_net_tran_id": "Net Transaction ID",
    "null_qty": "Quantity",
    "null_itm_fulfilment_id": "Item Fulfillment ID",
    "null_del_pc": "Delivery Pincode",
    "null_created_date_time": "Created Date",
    "null_domain": "Domain",
    "null_del_cty": "Delivery City",
    "null_cans_code": "Cancellation Code",
    "null_cans_dt_time": "Cancellation Date",
    "null_ord_stats": "Order Status",
    "null_fulfil_status": "Fulfillment Status",
    "null_itm_cat": "Item Category",
    "null_cat_cons": "Category",
    "null_sell_pincode": "Seller Pincode",
    "null_prov_id": "Provider ID",
    "null_itm_id": "Item ID",
    "null_sell_np": "Null Seller NP",
    "null_net_ord_id": "Network Order ID",
    "null_sell_cty": "Seller City",
    "total_orders" : "Total Orders",
    "total_canceled_orders" : "Cancelled Orders",
    "order_date": "Order Date"
}

## Top Cards 

### Total Orders with Delta

In [ ]:
dt_rng = bq.get_date_range()

max_date = dt_rng[1]

In [ ]:
prev_dt = max_date - timedelta(days=1)

In [ ]:
seller = "agg.dominos.co.in"

In [ ]:
prev_dt

In [ ]:
st_dt = datetime.strptime("2024-08-10","%Y-%m-%d")

In [ ]:
seller = None